In [272]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns

In [205]:
plt.rc("figure", figsize=(15,15))

In [116]:
df = pd.read_csv('heart.csv')

In [117]:
df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


Going to rename the columns to their longer, more descriptive forms.

In [118]:
df = df.rename(columns = {'exng' : "exercise induced agnia", 'caa': 'number of major vessels', 'cp' : 'chest pain type', 
                     'trtbps' : 'resting blood pressure',  'chol' : 'cholestoral', 'fbs': 'high fasting blood sugar',
                    'restecg' : 'resting_ecg', 'thalachh' : 'maximum heart rate'})

In [6]:
df.head()

,age,sex,chest pain type,resting blood pressure,cholestoral,high fasting blood sugar,resting ecg,maximum heart rate,exercise induced agnia,oldpeak,slp,number of major vessels,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [7]:
df.describe()

,age,sex,chest pain type,resting blood pressure,cholestoral,high fasting blood sugar,resting ecg,maximum heart rate,exercise induced agnia,oldpeak,slp,number of major vessels,thall,output
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


I want to simulate forcasting, so rather than divide these in train/test, I'm going to pretend we've been given data on patients with low heart rates and we want to predict what would happen as heart rate increases. A heart rate of 165 is chosen arbitrarily.

In [119]:
low_heart_rate = df[df['maximum heart rate'] <= 165]
high_heart_rate = df[df['maximum heart rate'] > 165]

In [120]:
low_heart_rate.corr()

,age,sex,chest pain type,resting blood pressure,cholestoral,high fasting blood sugar,resting_ecg,maximum heart rate,exercise induced agnia,oldpeak,slp,number of major vessels,thall,output
age,1.000000,-0.073164,0.056675,0.270067,0.141476,0.080432,-0.080991,-0.164052,-0.028928,0.150882,-0.067254,0.263307,0.005053,-0.060452
sex,-0.073164,1.000000,-0.055550,-0.125498,-0.246625,0.017609,-0.059297,-0.133238,0.184232,0.106322,-0.035782,0.124984,0.201201,-0.360547
chest pain type,0.056675,-0.055550,1.000000,0.072607,-0.029445,0.058127,0.020930,0.243226,-0.423920,-0.155190,0.115593,-0.225590,-0.137109,0.415650
resting blood pressure,0.270067,-0.125498,0.072607,1.000000,0.115438,0.187251,-0.108132,0.005708,0.056030,0.196544,-0.170477,0.064728,0.016480,-0.129095
cholestoral,0.141476,-0.246625,-0.029445,0.115438,1.000000,-0.002472,-0.133892,0.097276,0.040951,0.029300,0.036706,0.111390,0.088579,-0.022415
high fasting blood sugar,0.080432,0.017609,0.058127,0.187251,-0.002472,1.000000,-0.115250,0.042040,0.035570,-0.023329,-0.014543,0.139876,-0.069963,-0.032146
resting_ecg,-0.080991,-0.059297,0.020930,-0.108132,-0.133892,-0.115250,1.000000,-0.004915,-0.079687,-0.065242,0.088012,-0.081115,-0.027508,0.096457
maximum heart rate,-0.164052,-0.133238,0.243226,0.005708,0.097276,0.042040,-0.004915,1.000000,-0.291035,-0.259153,0.328394,-0.168526,-0.020197,0.328400
exercise induced agnia,-0.028928,0.184232,-0.423920,0.056030,0.040951,0.035570,-0.079687,-0.291035,1.000000,0.231515,-0.242891,0.115777,0.173897,-0.423002
oldpeak,0.150882,0.106322,-0.155190,0.196544,0.029300,-0.023329,-0.065242,-0.259153,0.231515,1.000000,-0.581175,0.252838,0.172478,-0.399146


We aren't seeing any multicollinearity in our training data.

In [115]:
low_heart_rate.describe()

,age,sex,chest pain type,resting blood pressure,cholestoral,high fasting blood sugar,resting ecg,maximum heart rate,exercise induced agnia,oldpeak,slp,number of major vessels,thall,output
count,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000
mean,56.573333,0.675556,0.835556,132.564444,249.053333,0.160000,0.502222,140.551111,0.404444,1.216889,1.297778,0.813333,2.355556,0.453333
std,8.211773,0.469211,1.028425,18.407262,53.667102,0.367423,0.535561,19.121284,0.491878,1.202074,0.601519,1.018051,0.659936,0.498927
min,35.000000,0.000000,0.000000,94.000000,131.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,51.000000,0.000000,0.000000,120.000000,212.000000,0.000000,0.000000,127.000000,0.000000,0.100000,1.000000,0.000000,2.000000,0.000000
50%,57.000000,1.000000,0.000000,130.000000,243.000000,0.000000,0.000000,144.000000,0.000000,1.000000,1.000000,0.000000,2.000000,0.000000
75%,62.000000,1.000000,2.000000,140.000000,278.000000,0.000000,1.000000,156.000000,1.000000,1.900000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,165.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [150]:
low_heart_rate.head()

,age,sex,chest pain type,resting blood pressure,cholestoral,high fasting blood sugar,resting_ecg,maximum heart rate,exercise induced agnia,oldpeak,slp,number of major vessels,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
5,57,1,0,140,192,0,1,148,0,0.4,1,0,1,1
6,56,0,1,140,294,0,0,153,0,1.3,1,0,2,1
8,52,1,2,172,199,1,1,162,0,0.5,2,0,3,1


In [12]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [126]:
res = smf.logit("""output ~ age + sex + C(Q("chest pain type")) + Q('resting blood pressure') 
+ cholestoral + Q('high fasting blood sugar') + C(resting_ecg) 
+  Q('maximum heart rate') + Q('exercise induced agnia') + Q('oldpeak') 
+ C(slp) +  Q('number of major vessels') + C(thall)""", data=low_heart_rate).fit()

Optimization terminated successfully.
         Current function value: 0.345533
         Iterations 7


In [127]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 output   No. Observations:                  225
Model:                          Logit   Df Residuals:                      205
Method:                           MLE   Df Model:                           19
Date:                Mon, 26 Jul 2021   Pseudo R-squ.:                  0.4983
Time:                        16:03:26   Log-Likelihood:                -77.745
converged:                       True   LL-Null:                       -154.98
Covariance Type:            nonrobust   LLR p-value:                 3.008e-23
=================================================================================================
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                         1.9093      3.708      0.515      0.607      -5.359       9.177
C(Q("chest pain type"))[T.1]      1.3515      0.711      1.901      0.057      -0.042       2.745
C(Q("chest pain type"))[T.2]      1.5027      0.519      2.897      0.004       0.486       2.520
C(Q("chest pain type"))[T.3]      2.0056      0.773      2.596      0.009       0.491       3.520
C(resting_ecg)[T.1]               0.2845      0.424      0.671      0.502      -0.546       1.115
C(resting_ecg)[T.2]              -0.8572      2.374     -0.361      0.718      -5.510       3.795
C(slp)[T.1]                      -1.0117      0.945     -1.070      0.284      -2.864       0.841
C(slp)[T.2]                      -0.2596      1.060     -0.245      0.807      -2.337       1.818
C(thall)[T.1]                     1.7777      2.523      0.704      0.481      -3.168       6.724
C(thall)[T.2]                     1.4746      2.436      0.605      0.545      -3.300       6.250
C(thall)[T.3]                     0.6445      2.450      0.263      0.792      -4.157       5.446
age                               0.0328      0.028      1.153      0.249      -0.023       0.089
sex                              -2.0740      0.589     -3.521      0.000      -3.229      -0.919
Q('resting blood pressure')      -0.0282      0.012     -2.270      0.023      -0.053      -0.004
cholestoral                      -0.0050      0.005     -1.107      0.268      -0.014       0.004
Q('high fasting blood sugar')     0.1883      0.650      0.290      0.772      -1.085       1.462
Q('maximum heart rate')           0.0175      0.013      1.326      0.185      -0.008       0.043
Q('exercise induced agnia')      -0.9105      0.473     -1.925      0.054      -1.838       0.017
Q('oldpeak')                     -0.4204      0.254     -1.652      0.099      -0.919       0.078
Q('number of major vessels')     -0.9937      0.281     -3.537      0.000      -1.544      -0.443
=================================================================================================
"""

Let's run f tests on the categorical parameters, rather than relying on their individual p values.

In [108]:
res.f_test('(C(Q("chest pain type"))[T.1] = 0), (C(Q("chest pain type"))[T.2] = 0), (C(Q("chest pain type"))[T.3] = 0)')

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=array([[3.99700281]]), p=0.008557203865961427, df_denom=205, df_num=3>

In [125]:
res.f_test('(C(resting_ecg)[T.1] = 0), (C(resting_ecg)[T.2] = 0)')

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=array([[0.31861219]]), p=0.7275169194600002, df_denom=205, df_num=2>

In [128]:
res.f_test('(C(slp)[T.1] = 0), (C(slp)[T.2] = 0)')

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=array([[1.60185022]]), p=0.20403526870974165, df_denom=205, df_num=2>

In [129]:
res.f_test('(C(thall)[T.1] = 0), (C(thall)[T.2] = 0), (C(thall)[T.3] = 0)')

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=array([[1.53446704]]), p=0.20669250940917208, df_denom=205, df_num=3>

We've got quite a few variables with very high p scores, let's remove them from the model.

In [277]:
res = smf.logit("""output ~ sex + Q('resting blood pressure') + C(Q("chest pain type"))
+  Q('maximum heart rate') + Q('exercise induced agnia') + Q('oldpeak') 
+  Q('number of major vessels')""", data=low_heart_rate).fit()

Optimization terminated successfully.
         Current function value: 0.374615
         Iterations 7


In [278]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 output   No. Observations:                  225
Model:                          Logit   Df Residuals:                      215
Method:                           MLE   Df Model:                            9
Date:                Tue, 27 Jul 2021   Pseudo R-squ.:                  0.4561
Time:                        14:26:01   Log-Likelihood:                -84.288
converged:                       True   LL-Null:                       -154.98
Covariance Type:            nonrobust   LLR p-value:                 5.361e-26
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                        2.5845      2.222      1.163      0.245      -1.770       6.939
C(Q("chest pain type"))[T.1]     1.6043      0.659      2.434      0.015       0.313       2.896
C(Q("chest pain type"))[T.2]     1.5001      0.483      3.108      0.002       0.554       2.446
C(Q("chest pain type"))[T.3]     2.1507      0.733      2.934      0.003       0.714       3.587
sex                             -1.9769      0.455     -4.341      0.000      -2.869      -1.084
Q('resting blood pressure')     -0.0245      0.011     -2.173      0.030      -0.047      -0.002
Q('maximum heart rate')          0.0183      0.012      1.503      0.133      -0.006       0.042
Q('exercise induced agnia')     -1.0885      0.443     -2.458      0.014      -1.956      -0.221
Q('oldpeak')                    -0.5144      0.212     -2.424      0.015      -0.930      -0.098
Q('number of major vessels')    -0.8779      0.238     -3.683      0.000      -1.345      -0.411
================================================================================================
"""

In [214]:
res = smf.logit("""output ~ sex + C(Q("chest pain type")) + Q('resting blood pressure') + Q('exercise induced agnia') + Q('oldpeak') 
+  Q('number of major vessels')""", data=low_heart_rate).fit()

Optimization terminated successfully.
         Current function value: 0.379842
         Iterations 7


In [215]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 output   No. Observations:                  225
Model:                          Logit   Df Residuals:                      216
Method:                           MLE   Df Model:                            8
Date:                Tue, 27 Jul 2021   Pseudo R-squ.:                  0.4485
Time:                        13:04:10   Log-Likelihood:                -85.464
converged:                       True   LL-Null:                       -154.98
Covariance Type:            nonrobust   LLR p-value:                 3.785e-26
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                        5.0245      1.576      3.187      0.001       1.935       8.114
C(Q("chest pain type"))[T.1]     1.6672      0.647      2.575      0.010       0.398       2.936
C(Q("chest pain type"))[T.2]     1.6130      0.479      3.370      0.001       0.675       2.551
C(Q("chest pain type"))[T.3]     2.1830      0.730      2.989      0.003       0.751       3.614
sex                             -1.9730      0.457     -4.321      0.000      -2.868      -1.078
Q('resting blood pressure')     -0.0225      0.011     -2.032      0.042      -0.044      -0.001
Q('exercise induced agnia')     -1.1944      0.431     -2.771      0.006      -2.039      -0.350
Q('oldpeak')                    -0.5803      0.211     -2.753      0.006      -0.994      -0.167
Q('number of major vessels')    -0.8988      0.235     -3.830      0.000      -1.359      -0.439
================================================================================================
"""

We do lose a small amount of R2, but this is to be expected whenever we remove variables.

In [30]:
predicted = res.predict(high_heart_rate)

In [58]:
high_heart_rate = high_heart_rate.merge(predicted.rename('predicted'), left_index=True, right_index=True)

In [59]:
high_heart_rate['predicted_cat'] = round(high_heart_rate.predicted)

In [60]:
high_heart_rate.head()

,age,sex,chest pain type,resting blood pressure,cholestoral,high fasting blood sugar,resting ecg,maximum heart rate,exercise induced agnia,oldpeak,slp,number of major vessels,thall,output,predicted_cat,predicted
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1,1.0,0.617772
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1,1.0,0.958638
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1,1.0,0.870553
7,44,1,1,120,263,0,1,173,0,0.0,2,0,3,1,1.0,0.842412
9,57,1,2,150,168,0,1,174,0,1.6,2,0,2,1,1.0,0.803508


In [61]:
from sklearn.metrics import accuracy_score

'{0:.0%}'.format(accuracy_score(high_heart_rate.output, round(predicted)))

'82%'

Far from ideal but not bad for such a simple model.

In [62]:
high_heart_rate[high_heart_rate.output != high_heart_rate.predicted_cat].describe()

,age,sex,chest pain type,resting blood pressure,cholestoral,high fasting blood sugar,resting ecg,maximum heart rate,exercise induced agnia,oldpeak,slp,number of major vessels,thall,output,predicted_cat,predicted
count,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000
mean,50.071429,0.857143,1.000000,137.714286,236.714286,0.142857,0.500000,175.071429,0.142857,0.278571,1.714286,1.357143,2.285714,0.500000,0.500000,0.556829
std,8.552681,0.363137,1.037749,19.718904,43.137399,0.363137,0.518875,7.650684,0.363137,0.563203,0.611250,1.645841,0.468807,0.518875,0.518875,0.240798
min,38.000000,0.000000,0.000000,110.000000,175.000000,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.260141
25%,43.250000,1.000000,0.000000,130.000000,211.000000,0.000000,0.000000,169.500000,0.000000,0.000000,2.000000,0.000000,2.000000,0.000000,0.000000,0.324116
50%,51.500000,1.000000,1.000000,138.000000,232.500000,0.000000,0.500000,173.000000,0.000000,0.000000,2.000000,1.000000,2.000000,0.500000,0.500000,0.516546
75%,56.250000,1.000000,2.000000,139.500000,257.500000,0.000000,1.000000,176.250000,0.000000,0.000000,2.000000,2.500000,2.750000,1.000000,1.000000,0.735588
max,65.000000,1.000000,3.000000,192.000000,330.000000,1.000000,1.000000,195.000000,1.000000,1.500000,2.000000,4.000000,3.000000,1.000000,1.000000,0.960094


In [63]:
high_heart_rate.describe()

,age,sex,chest pain type,resting blood pressure,cholestoral,high fasting blood sugar,resting ecg,maximum heart rate,exercise induced agnia,oldpeak,slp,number of major vessels,thall,output,predicted_cat,predicted
count,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000
mean,48.000000,0.705128,1.346154,128.910256,238.217949,0.115385,0.602564,175.884615,0.102564,0.528205,1.692308,0.487179,2.192308,0.807692,0.807692,0.746973
std,8.488342,0.458936,0.951102,14.513931,45.490872,0.321553,0.492535,7.413538,0.305352,0.852635,0.565403,1.003159,0.428155,0.396664,0.396664,0.265370
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,166.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.104855
25%,41.250000,0.000000,1.000000,120.000000,205.500000,0.000000,0.000000,170.250000,0.000000,0.000000,1.250000,0.000000,2.000000,1.000000,1.000000,0.626025
50%,47.500000,1.000000,1.000000,130.000000,235.500000,0.000000,1.000000,173.500000,0.000000,0.000000,2.000000,0.000000,2.000000,1.000000,1.000000,0.871474
75%,54.000000,1.000000,2.000000,138.000000,266.750000,0.000000,1.000000,179.750000,0.000000,0.875000,2.000000,0.000000,2.000000,1.000000,1.000000,0.944802
max,67.000000,1.000000,3.000000,192.000000,342.000000,1.000000,1.000000,202.000000,1.000000,3.800000,2.000000,4.000000,3.000000,1.000000,1.000000,0.984314


In [64]:
len(high_heart_rate[high_heart_rate.output != high_heart_rate.predicted_cat])

14

At a glance it looks like the model is performing poorly for patients in high numbers of major blood vessles.

In [65]:
len(high_heart_rate[(high_heart_rate['number of major vessels'] > 0) & (high_heart_rate['output'] != high_heart_rate['predicted_cat'])])

8

They make up over half of our incorrect predictions.

In [340]:
high_heart_rate[(high_heart_rate['number of major vessels'] > 0) & (high_heart_rate['output'] != high_heart_rate['predicted_cat'])]

,age,sex,chest pain type,resting blood pressure,cholestoral,high fasting blood sugar,resting ecg,maximum heart rate,exercise induced agnia,oldpeak,slp,number of major vessels,thall,output,predicted,predicted_cat
92,52,1,2,138,223,0,1,169,0,0.0,2,4,2,1,0.156094,0.0
99,53,1,2,130,246,1,0,173,0,0.0,2,3,2,1,0.335193,0.0
100,42,1,3,148,244,0,0,178,0,0.8,2,2,2,1,0.470099,0.0
163,38,1,2,138,175,0,1,173,0,0.0,2,4,2,1,0.167679,0.0
164,38,1,2,138,175,0,1,173,0,0.0,2,4,2,1,0.167679,0.0
200,44,1,0,110,197,0,0,177,0,0.0,2,1,2,0,0.774758,1.0
222,65,1,3,138,282,1,0,174,0,1.4,1,1,2,0,0.590946,1.0
248,54,1,1,192,283,0,0,195,0,0.0,2,1,3,0,0.722120,1.0
302,57,0,1,130,236,0,0,174,0,0.0,1,1,2,0,0.936073,1.0


In [66]:
lhr_vessels = len(low_heart_rate[low_heart_rate['number of major vessels'] > 0]) / len(low_heart_rate)
hhr_vessels = len(high_heart_rate[high_heart_rate['number of major vessels'] > 0]) / len(high_heart_rate)
lhr_attack_rate = len(low_heart_rate[(low_heart_rate['number of major vessels'] > 0) & (low_heart_rate.output == 1)]) / len(low_heart_rate)
hhr_attack_rate = len(high_heart_rate[(high_heart_rate['number of major vessels'] > 0) & (high_heart_rate.output == 1)]) / len(high_heart_rate)
print(f"The number of vessels present in low heart patients is a lot higher than in the number of high heart rate patients: {lhr_vessels:.0%} vs {hhr_vessels:.0%}. \nMaybe there's some condition among people with large numbers of blood vessels we're missing.")

The number of vessels present in low heart patients is a lot higher than in the number of high heart rate patients: 48% vs 24%. 
Maybe there's some condition among people with large numbers of blood vessels we're missing.


In [67]:
from sklearn.metrics import classification_report
print(classification_report(high_heart_rate.output, round(predicted)))

              precision    recall  f1-score   support

           0       0.53      0.53      0.53        15
           1       0.89      0.89      0.89        63

    accuracy                           0.82        78
   macro avg       0.71      0.71      0.71        78
weighted avg       0.82      0.82      0.82        78



Looks like most of room for improve is in false positives. We have a lot of cases where we're predicting heart attack in patients that will not experience them.